In [ ]:
"""
Validation of open-gira power outage model
This notebook compares modelled and observed customer disconnected estimates
Observed data are collated media and goverment reports (CSV files in validation/)
Modelled data are exposure_by_country.nc for the whole IBTrACS storm set
"""

In [ ]:
import os
from glob import glob

import geopandas as gpd
import matplotlib
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# model outputs
root_path = "/home/fred/projects/open_gira/open-gira/"
tracks_path = os.path.join(root_path, "results/input/IBTrACS/processed/v4.geoparquet")
targets_path = os.path.join(root_path, "results/power/targets_with_iso_a3.geoparquet")
exposure_path = os.path.join(root_path, "results/power/by_storm_set/IBTrACS/exposure_by_country.nc")

# validation data
# https://www.un.org/development/desa/pd/data/household-size-and-composition
household_path = os.path.join(root_path, "undesa_pd_2022_hh-size-composition.xlsx")
# https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv
iso_codes = os.path.join(root_path, "iso_codes.csv")
# see source column
outage_paths = glob(os.path.join(root_path, "validation/observed_outages/*.csv"))

In [ ]:
# read iso numeric to alpha table
rename_dict = {
    "Country": "name",
    "Alpha-2 code": "iso_a2",
    "Alpha-3 code": "iso_a3",
    "Numeric code": "iso_num"
}
iso = pd.read_csv(iso_codes, usecols=rename_dict.keys())
iso = iso.rename(columns=rename_dict)
# strip out quotes
for c in iso.columns:
    iso[c] = iso[c].str.replace('"', "").str.strip()
iso.iso_num = iso.iso_num.astype(int)

In [ ]:
# read household size data
rename_dict = {
    "ISO Code": "iso_num",
    "Reference date (dd/mm/yyyy)": "ref_date",
    "Average household size (number of members)": "mean_household_pop"
}
hh = pd.read_excel(household_path, sheet_name="HH size and composition 2022", header=4, usecols=rename_dict.keys())
hh = hh.rename(columns=rename_dict)
hh.ref_date = pd.to_datetime(hh.ref_date, dayfirst=True)

# any non-numeric value -> NaN (some '..' placeholders in excel data)
hh["mean_household_pop"] = pd.to_numeric(hh["mean_household_pop"], errors="coerce")

# drop any rows without a valid mean household pop
hh = hh[~hh.mean_household_pop.isna()]

# discard all but most recent entry for each territory
hh = hh.sort_values(["ref_date"], ascending=False).drop_duplicates("iso_num", keep="first")

# merge in iso alpha ids (will permit join with outage data)
hh = hh.merge(iso, how="left", on="iso_num")

In [ ]:
# now compare population_affected with the exposure_by_country.nc data
ds = xr.open_dataset(exposure_path).rename({"country": "country_iso_a3"})

by_threshold = {}
for threshold in ds.threshold.values:

    # storm loop
    obs_data = {os.path.basename(p).split(".")[0]: pd.read_csv(p) for p in sorted(outage_paths)}
    by_storm = {}
    for event_id, obs in obs_data.items():
        
        try:
            # select model data for given storm and threshold
            mod = ds.customers_affected.sel(dict(event_id=event_id, threshold=threshold)).to_pandas()
        except KeyError:
            # storm has no modelled results for any threshold
            continue
        
        # create a duplicate dataframe for modelled and derived measures (as well as obs)
        by_storm[event_id] = obs.copy()
        
        # bring in demographic data to obs dataframe
        obs = obs.rename(columns={"country_iso_a3": "iso_a3"}).merge(hh, how="left", on="iso_a3")

        for i in range(len(obs)):
            # gap-fill the population_affected numbers where we have customers_affected only
            if pd.isna(obs.loc[i, "population_affected"]):
                by_storm[event_id].loc[i, "population_affected"] = \
                    obs.loc[i, "customers_affected"] * obs.loc[i, "mean_household_pop"]

        # take the largest estimate for each country (we assume reports underestimate outages)
        by_storm[event_id] = by_storm[event_id].groupby("country_iso_a3").max()
        by_storm[event_id] = by_storm[event_id][["population_affected"]]
        by_storm[event_id] = by_storm[event_id].rename(columns={"population_affected": "observed"})

        # modelled customers_affected (really population affected, should change this name!)
        mod.name = "modelled"
        by_storm[event_id] = by_storm[event_id].join(mod)
        by_storm[event_id]["error"] = \
            by_storm[event_id].modelled - by_storm[event_id].observed
        by_storm[event_id]["error_abs"] = \
            np.abs(by_storm[event_id]["error"])
        by_storm[event_id]["ratio"] = \
            by_storm[event_id].modelled / by_storm[event_id].observed
        by_storm[event_id]["error_norm"] = \
            by_storm[event_id].error / by_storm[event_id].observed
        by_storm[event_id]["signed_ratio"] = \
            np.sign(by_storm[event_id].error) * by_storm[event_id].ratio

    concat = pd.concat(by_storm)
    concat.index.names = ["event_id", "country_iso_a3"]
    by_threshold[threshold] = concat

# all data with a 3 level multi-index
data = pd.concat(by_threshold, names=["threshold", "event_id", "country_iso_a3"])

# write to disk
data.to_csv("outage_model_validation.csv")

In [ ]:
# example data for a given threshold
data.loc[32.5, :, :]

In [ ]:
# where are these tracks?
borders = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
tracks = gpd.read_parquet(tracks_path)
tracks = tracks[tracks.track_id.isin(set(data.index.get_level_values("event_id")))]

f, ax = plt.subplots(figsize=(16, 7))

# plot landmasses and political borders
borders.plot(ax=ax, facecolor="none", edgecolor="grey", alpha=0.5)

# plot tracks with colourbar for wind speed intensity
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.15)
markersize = np.exp(tracks.category)
tracks.plot(column="max_wind_speed_ms", ax=ax, s=markersize, cax=cax, legend=True, cmap="viridis")
cax.set_ylabel("Max wind speed $[ms^{-1}]$")

ax.set_xlim(-130, 180)
ax.set_ylim(-60, 75)
ax.set_title("Tracks of storms with observed outage data")
ax.set_xlabel("Longitude [deg]")
ax.set_ylabel("Latitude [deg]")

# second legend for saffir simpson storm classification
saffir_simpson = ax.legend(
    handles=[
        Line2D(
            [],
            [],
            color=plt.get_cmap("viridis")(0.8),
            lw=0,
            marker="o",
            markersize=np.sqrt(b),
            label=np.log(b),
        )
        for b in sorted(set(markersize))
    ],
    loc="lower left",
    title="Saffir-Simpson scale"
)

In [ ]:
def categorical_cmap(cmap_name: str, categories: list) -> dict[str: tuple[float]]:
    scalar_mappable = matplotlib.cm.ScalarMappable(matplotlib.colors.Normalize(0, 1), cmap_name)
    return {code: scalar_mappable.to_rgba(i / len(categories)) for i, code in enumerate(categories)}

# create some colourmaps and persist between plots

# make a country colormap
countries = sorted(set(data.index.get_level_values("country_iso_a3").values))
country_cmap = categorical_cmap("tab20c", countries)

# make a cmap for wind speed thresholds
thresholds = sorted(set(data.index.get_level_values("threshold").values))
wind_cmap = categorical_cmap("plasma", thresholds)

In [ ]:
# scatter plot modelled vs. observed by threshold

n = len(thresholds)
ncols = min([n, 4])
nrows = (n // ncols) + (n % ncols)
f, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(3 + 2.5 * ncols, 2 + 2 * nrows), squeeze=False)

for row in range(nrows):
    for col in range(ncols):

        i = col + (row * ncols)
        ax = axes[row, col]
        
        if i < n:
            threshold = thresholds[i]
            df = data.loc[threshold, :, :]

            # build a list of RGBA values the same length as the data
            colours = [country_cmap[value] for value in df.index.get_level_values("country_iso_a3").values]

            ax.scatter(df.observed, df.modelled, marker="x", c=colours)
            x = np.linspace(0, 10 * max(df.observed), 100)
            ax.plot(x, x, ls="--", c="cornflowerblue")
            ax.grid()
            ax.set_title(f"$s_{{th}} = {threshold} \ m s^{{-1}}$", size=10)
            ax.set_xscale("log")
            ax.set_yscale("log")
            ax.set_xlim(1E5, 1E8)
            ax.set_ylim(1E5, 1E8)

        # disable any axes we don't need
        else:
            ax.set_axis_off()

handles = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=v, label=k, markersize=8)
    for k, v in country_cmap.items() if isinstance(k, str)
]
f.legend(handles=handles, ncol=1, bbox_to_anchor=(0.95, 0.92), fontsize=9)
    
f.supxlabel("Observed pop. disconnected, $p_{d,obs}$")
f.supylabel("Modelled pop. disconnected, $p_{d,mod}$")
f.suptitle("Population disconnected, $p_{d}(s_{th})$")

plt.subplots_adjust(bottom=0.11, top=0.88, left=0.08, right=0.9, hspace=0.35, wspace=0.35)

In [ ]:
# scatter plot modelled vs. observed by threshold (same plot)

f, ax = plt.subplots(figsize=(6, 6))
    
for threshold in thresholds:
    df = data.loc[threshold, :, :]
    ax.scatter(df.observed, df.modelled, marker="x", label=threshold, color=wind_cmap[threshold])
    
x = np.linspace(0, 10 * max(data.observed), 100)
ax.plot(x, x, ls="--", c="cornflowerblue")
ax.grid()
ax.set_title("Population disconnected, $p_{d}(s_{th})$")
ax.set_xlabel("Observed pop. disconnected, $p_{d,obs}$")
ax.set_ylabel("Modelled pop. disconnected, $p_{d,mod}$")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(1E4, 1E8)
ax.set_ylim(1E4, 1E8)
ax.legend(title="Wind speed\n threshold,\n$s_{th}$, $[m s^{{-1}}]$")

In [ ]:
# scatter plot normalised residual as a function of threshold

df = data["error_norm"].reset_index(["threshold", "country_iso_a3"])
colours = [country_cmap[value] for value in df.country_iso_a3.values]

f, ax = plt.subplots(figsize=(8,6))
ax.scatter(df.threshold, df.error_norm, c=colours, label="Country", marker="x")
handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=v, label=k, markersize=8) for k, v in country_cmap.items() if isinstance(k, str)]
ax.legend(title='Country', handles=handles, fontsize=9, loc="upper right")
ax.grid()
ax.axhline(0, ls="--", c="cornflowerblue")
ax.set_xlabel(r"Wind speed threshold, $s_{th}$, $[m s^{-1}]$")
ax.set_ylabel(r"Normalised residual, $\mathrm{\frac{p_{d,mod} - p_{d,obs}}{p_{d,obs}}}$")
ax.set_title("Population disconnected, $p_{d}(s_{th})$, residuals")
ax.set_ylim(-1.5, 4)

plt.subplots_adjust(right=0.8)

# N.B. a couple of values are more more than 400% out, therefore out of plot area
# -1, worst possible score (model 100% underestimated)
# 0, perfect score
# + valued, progressively greater model overestimate

# N.B. our residuals, i.e. (mod - obs) / obs are not symmetric about 0 (-1 is the worst possible score)
# but we plot this measure as a) it's normalised, so can compare across scales
# and b) something like sign(mod - obs) * mod / obs, would give a perfect 0 for mod = 0  

In [ ]:
# as above, but only plot each storm-country pair once, at the threshold with minimum absolute error

measure = "error_norm"

results = []
for event_iso, df in data.groupby(["event_id", "country_iso_a3"]):
    min_error_mask = df.error_abs == df.error_abs.min()
    try:
        # only store the threshold, error pair if there's only a single minima
        threshold, = df[min_error_mask].index.get_level_values("threshold")
        statistic, = df.loc[min_error_mask, measure]
        results.append((threshold, *event_iso, statistic))
    except ValueError:
        continue
        
df = pd.DataFrame(results, columns=["threshold", "event_id", "country_iso_a3", measure])

f, ax = plt.subplots(figsize=(6,4))
ax.scatter(
    df.threshold,
    df.error_norm,
    c=[country_cmap[value] for value in df.country_iso_a3]
)
ax.set_ylabel(r"Normalised residual, $\mathrm{\frac{p_{d,mod}(s_{th}) - p_{d,obs}}{p_{d,obs}}}$")
ax.set_xlabel(r"Wind speed threshold, $s_{th}$, $[m s^{-1}]$")
ax.set_title("Population disconnected, $p_{d}$, residuals,\nfor threshold, $s_{th}$, with $min(abs(p_{d,mod}-p_{d,obs}))$")
ax.set_ylim(-1.1, 1)
ax.set_xticks(thresholds)
ax.grid()
handles = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=v, label=k, markersize=8)
    for k, v in country_cmap.items() if isinstance(k, str)
]
ax.legend(handles=handles, ncol=int(len(handles) / 2), fontsize=7, loc="lower right")

In [ ]:
def plot_residual_distribution_by_threshold(*, measure: str, x_label: str, x_min: float, x_max: float, bin_width: float, target: float = None):
    n = len(thresholds)
    ncols = min([n, 4])
    nrows = (n // ncols) + (n % ncols)
    f, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(3 + 2.5 * ncols, 2 + 2.25 * nrows), squeeze=False)
    
    n_bins = int((x_max - x_min) / bin_width + 1)
    bins = np.linspace(x_min, x_max, n_bins)

    max_ylim = 0
    for row in range(nrows):
        for col in range(ncols):

            i = col + (row * ncols)
            ax = axes[row, col]
            if i < n:
                threshold = thresholds[i]
                residual = data.loc[threshold, :, :][measure]

                ax.grid()

                # distribution
                ax.hist(
                    residual,
                    bins=bins,
                    alpha=0.5,
                    label="Dist.",
                    facecolor=wind_cmap[threshold]
                )

                # rug plot
                ax.plot(residual, np.zeros(len(residual)), 'b|', ms=15, alpha=0.5, label="Data")

                # average measures
                mean = np.nanmean(residual)
                median = np.nanmedian(residual)
                ax.axvline(median, ls="--", c="red", label=r"$p_{50}$")
                ax.axvline(mean, ls="-", c="green", label=r"$\mu$")

                # target
                if target is not None:
                    ax.axvline(0, ls="--", c="cornflowerblue", label="Target")

                ax.set_xlim(x_min, x_max)
                _, max_y = ax.get_ylim()
                max_ylim = max([max_ylim, max_y])

                ax.set_title(f"$s_{{th}} = {threshold} \ m s^{{-1}}$", size=10)
                props = dict(boxstyle='round', facecolor='wheat', alpha=0.35)
                ax.text(
                    0.95,
                    0.95,
                    "\n".join([f"$p_{{50}}$ = {median:.2f}", f"$\mu$ = {mean:.2f}"]),
                    transform=ax.transAxes,
                    fontsize=8,
                    verticalalignment='top',
                    horizontalalignment='right',
                    bbox=props
                )

            # disable any axes we don't need
            else:
                ax.set_axis_off()

    # set a common y range for all subplots
    for row in range(nrows):
        for col in range(ncols):
            i = col + (row * ncols)
            ax = axes[row, col]
            if i < n:
                ax.set_ylim(0, max_ylim)

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    f.legend(by_label.values(), by_label.keys(), ncol=5, bbox_to_anchor=(0.68, 0.93))

    f.supxlabel(x_label)
    f.supylabel("Frequency")
    f.suptitle("Population disconnected, $p_{d}(s_{th})$, residuals")

    plt.subplots_adjust(bottom=0.1, top=0.82, left=0.08, right=0.87, hspace=0.3, wspace=0.25)

In [ ]:
# plot residual distributions by threshold
plot_residual_distribution_by_threshold(
    measure="error_norm",
    x_label=r"Normalised residual, $\mathrm{\frac{p_{d,mod} - p_{d,obs}}{p_{d,obs}}}$",
    x_min=-1,
    x_max=4,
    bin_width=2/5,
    target=0
)